In [1]:
!wget --no-check-certificate https://storage.googleapis.com/emcassavadata/cassavaleafdata.zip -O ./data/cassavaleafdata.zip
!unzip ./data/cassavaleafdata.zip -d ./data
!rm ./data/cassavaleafdata.zip

--2024-12-11 01:04:51--  https://storage.googleapis.com/emcassavadata/cassavaleafdata.zip
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
2404:6800:4005:81f::201b, 2404:6800:4005:821::201b, 2404:6800:4005:80b::201b, ...
connected. to storage.googleapis.com (storage.googleapis.com)|2404:6800:4005:81f::201b|:443... 
HTTP request sent, awaiting response... 200 OK
Length: 1354096203 (1.3G) [application/octet-stream]
Saving to: ‘./data/cassavaleafdata.zip’

./data/cassavaleafd 100%[===================>]   1.26G  25.3MB/s    in 49s     

2024-12-11 01:05:41 (26.4 MB/s) - ‘./data/cassavaleafdata.zip’ saved [1354096203/1354096203]

Archive:  ./data/cassavaleafdata.zip
   creating: ./data/cassavaleafdata/
   creating: ./data/cassavaleafdata/test/
   creating: ./data/cassavaleafdata/test/cbb/
  inflating: ./data/cassavaleafdata/test/cbb/test-cbb-0.jpg  
  inflating: ./data/cassavaleafdata/test/cbb/test-cbb-1.jpg  
  inflating: ./data/cassavaleafdata/test/cbb/test-cbb-10.jpg  
  inflat

In [2]:
import os
import time
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from torchinfo import summary

import matplotlib.pyplot as plt
from PIL import Image

In [3]:
data_paths = {
    'train': './data/cassavaleafdata/train',
    'valid': './data/cassavaleafdata/validation/',
    'test': './data/cassavaleafdata/test/'
}

In [4]:
def loader(path):
    return Image.open(path)

In [5]:
img_size=150
train_transforms = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor()
])

train_data = datasets.ImageFolder(
    root=data_paths['train'],
    loader=loader,
    transform=train_transforms
)
valid_data = datasets.ImageFolder(
    root=data_paths['valid'],
    loader=loader,
    transform=train_transforms
)
test_data = datasets.ImageFolder(
    root=data_paths['test'],
    loader=loader,
    transform=train_transforms
)

BATCH_SIZE = 256

train_dataloader = data.DataLoader(
    train_data,
    shuffle=True,
    batch_size=BATCH_SIZE
)
valid_dataloader = data.DataLoader(
    valid_data,
    batch_size=BATCH_SIZE
)

In [6]:
class LeNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super(LeNetClassifier, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, padding='same')
        self.avgpool1 = nn.AvgPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.avgpool2 = nn.AvgPool2d(kernel_size=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(16 * 35 * 35, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.avgpool1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.avgpool2(x)
        x = F.relu(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [7]:
def train(model, optimizer, criterion, train_dataloader, device, epoch=0, log_interval=50):
    model.train()
    total_acc, total_count = 0, 0
    losses = []
    start_time = time.time()

    for idx, (inputs, labels) in enumerate(train_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        predictions = model(inputs)

        loss = criterion(predictions, labels)
        losses.append(loss.item())

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predictions.argmax(1) == labels).sum().item()
        total_count += labels.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(
                "| Epoch {:3d} | {:5d}/{:5d} batches "
                "| Accuracy {:8.3f}".format(
                    epoch, idx, len(train_dataloader), total_acc / total_count
                )
            )
            total_acc, total_count = 0, 0
            start_time = time.time()

    epoch_acc = total_acc / total_count
    epoch_loss = sum(losses) / len(losses)
    return epoch_acc, epoch_loss

In [8]:
def evaluate(model, criterion, valid_dataloader):
    model.eval()
    total_acc, total_count = 0, 0
    losses = []

    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(valid_dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            predictions = model(inputs)

            loss = criterion(predictions, labels)
            losses.append(loss.item())

            total_acc += (predictions.argmax(1) == labels).sum().item()
            total_count += labels.size(0)

    epoch_acc = total_acc / total_count
    epoch_loss = sum(losses) / len(losses)
    return epoch_acc, epoch_loss

In [9]:
num_classes = len(train_data.classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

lenet_model = LeNetClassifier(num_classes)
lenet_model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(lenet_model.parameters())

num_epochs = 10
save_model = './model'

train_accs, train_losses = [], []
eval_accs, eval_losses = [], []
best_loss_eval = 100

for epoch in range(1, num_epochs+1):
    epoch_start_time = time.time()

    train_acc, train_loss = train(lenet_model, optimizer, criterion, train_dataloader, device, epoch)
    train_accs.append(train_acc)
    train_losses.append(train_loss)

    eval_acc, eval_loss = evaluate(lenet_model, criterion, valid_dataloader)
    eval_accs.append(eval_acc)
    eval_losses.append(eval_loss)

    if eval_loss < best_loss_eval:
        torch.save(lenet_model.state_dict(), save_model + '/lenet_model.pt')

    print("-" * 59)
    print(
        "| End of epoch {:3d} | Time: {:5.2f}s | Train Accuracy {:8.3f} | Train Loss {:8.3f} "
        "| Valid Accuracy {:8.3f} | Valid Loss {:8.3f} ".format(
            epoch, time.time() - epoch_start_time, train_acc, train_loss, eval_acc, eval_loss
        )
    )
    print("-" * 59)

-----------------------------------------------------------
| End of epoch   1 | Time: 68.49s | Train Accuracy    0.467 | Train Loss    1.358 | Valid Accuracy    0.473 | Valid Loss    1.432 
-----------------------------------------------------------
-----------------------------------------------------------
| End of epoch   2 | Time: 66.31s | Train Accuracy    0.470 | Train Loss    1.324 | Valid Accuracy    0.482 | Valid Loss    1.406 
-----------------------------------------------------------
-----------------------------------------------------------
| End of epoch   3 | Time: 67.37s | Train Accuracy    0.488 | Train Loss    1.282 | Valid Accuracy    0.497 | Valid Loss    1.381 
-----------------------------------------------------------
-----------------------------------------------------------
| End of epoch   4 | Time: 66.88s | Train Accuracy    0.509 | Train Loss    1.240 | Valid Accuracy    0.528 | Valid Loss    1.397 
--------------------------------------------------------

In [10]:
lenet_model.load_state_dict(torch.load(save_model + "/lenet_model.pt", weights_only=True))
lenet_model.eval()

LeNetClassifier(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1), padding=same)
  (avgpool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (avgpool2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=19600, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=5, bias=True)
)

In [11]:
test_dataloader = data.DataLoader(
    test_data,
    batch_size=BATCH_SIZE
)
test_acc, test_loss = evaluate(lenet_model, criterion, test_dataloader)
test_acc, test_loss

(0.5570291777188329, 1.2441428750753403)